# Transmission FTIR Spectra

- This Jupyter notebook provides an example workflow for processing transmission FTIR spectra through PyIRoGlass. 

- The Jupyter notebook and data can be accessed here: https://github.com/SarahShi/PyIRoGlass/blob/main/docs/examples/transmission_ftir/. 

- You need to have the PyIRoGlass PyPi package on your machine once. If you have not done this, please uncomment (remove the #) symbol and run the cell below. 

In [ ]:
#!pip install PyIRoGlass

# Load Python Packages and Data

## Load Python Packages

In [ ]:
# Import packages

import PyIRoGlass as pig

from IPython.display import Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pig.__version__

## Set paths to data

Update the path to the directory containing the spectra, as well as the paths to the chemistry and thickness data.

In [ ]:
spectrum_path = 'SPECTRA/'
print(spectrum_path)

chemistry_thickness_path = 'ChemThick.csv'
print(chemistry_thickness_path)

## Set desired output file directory name

Update the `export_path` to the desired output location.

In [ ]:
export_path = 'RESULTS'
print(export_path)

## Load transmission FTIR spectra along with chemistry thickness data

We will use the class `pig.SampleDataLoader` to load all FTIR spectra and chemistry thickness data. The class takes the arguments: 

- `spectrum_path`: String or list path to the directory with spectral data
- `chemistry_thickness_path`: String path to CSV file with glass chemistry and thickness data

and contains the methods: 

- `load_spectrum_directory`: Loads spectral data
- `load_chemistry_thickness`: Loads chemistry thickness data
- `load_all_data`: Loads spectral and chemistry thickness data

Here, we use `load_all_data`. This returns the outputs of: 

- `dfs_dict`: Dictionary where the keys are file identifiers and values are DataFrames with spectral data
- `chemistry`: DataFrame of chemical data
- `thickness`: DataFrame of thickness data

The file names from the spectra (what comes before the .CSV) are important when we load in melt compositions and thicknesses. Unique identifiers identify the same samples. Make sure that this `ChemThick.CSV` file has the same sample names as the loaded spectra.

In [ ]:
loader = pig.SampleDataLoader(spectrum_path=spectrum_path, chemistry_thickness_path=chemistry_thickness_path)
dfs_dict, chemistry, thickness = loader.load_all_data()

Let's look at what `dfs_dict`, a dictionary of transmission FTIR spectra, looks like. Samples are identified by their file names (keys) and the wavenumber and absorbance data are stored as dataframes for each spectrum (values).

In [ ]:
dfs_dict

Display `chemistry`, the DataFrame of glass compositions.

In [ ]:
chemistry

Display `thickness`, the DataFrame of wafer thicknesses.

In [ ]:
thickness

See that the sample names of the spectra in `dfs_dict`, `chemistry`, and `thickness` all align. 

# We're ready to roll -- MCMC, here we come! 

We use the function `pig.calculate_baselines`, which takes in two arguments: 

- `dfs_dict`: Dictionary where the keys are file identifiers and values are DataFrames with spectral data
- `export_path`: Desired output directory name, or `None` to prevent figure generation

and returns: 

- `Volatile_PH`: DataFrame with peak heights and their associated uncertainties
- `failures`: List of file identifiers for which analysis failed

Running this code will take a few seconds to minutes per spectra, as it is fitting $\mathrm{10^6}$ baselines and peaks to your spectrum to sample uncertainty. If any samples fail, they will be returned in the list `failures`. It took 20 seconds to process 1 spectrum on my M2 Macbook Pro with 12 CPU cores. The same task takes about 2 minutes on Google Colab. 

The function automatically saves this file as a CSV, so you have this information. We will also use this DataFrame to calculate concentrations. 

In [ ]:
Volatile_PH, failures = pig.calculate_baselines(dfs_dict, export_path)

`pig.calculate_baselines` returns `Volatile_PH`, a DataFrame of the output peak heights and associated uncertainties. Let's see what is included. 

In [ ]:
Volatile_PH

We can look at all the columns in this DataFrame, given the size. 

In [ ]:
Volatile_PH.columns

All columns with the prefix of `PH` represent a peak height. All columns with the suffix of `_M` represent the mean value, and the suffix of `_STD` represents 1 $\sigma$. 

The column `H2Ot_3550_SAT` returns a `-` if the sample is not saturated, and a `*` if the sample is saturated. This is based on the maximum absorbance of the peak, and the warning of `*` indicates that we must consider the concentrations more. The following functions calculating concentration handle this and will suggest best values to use. 

The columns `STN_P5200` and `STN_P4500` represent the signal to noise ratios for the $\mathrm{H_2O_{m,5200}}$ and $\mathrm{OH^-_{4500}}$ peaks. If the values are greater than 4, indicating that the signal is meaningful, the ERR_5200 and ERR_4500 peaks return a `-` value. If signal-to-noise is too low, the warning of `*` is returned. 

The columns after describe the fitting parameters for generating the baseline and the $\mathrm{H_2O_{m,1635}}$ peak, so you can generate the baseline yourself. 

# Outputs

Quite few figures, log files, and npz files are generated by `pig.calculate_baselines`, assuming you provide an export path. Let's look at a few of them together. 

PyIRoGlass creates this figure for visualizing how each peak within the 1000-5500 cm $\mathrm{^{-1}}$ is fit, with their peak heights shown. 

In [ ]:
Image("https://github.com/sarahshi/PyIRoGlass/raw/main/docs/_static/AC4_OL49_021920_30x30_H2O_a.png")

We can visualize how well PyIRoGlass does in fitting this transmission FTIR spectrum, with the modelfit figure. This plots the fit from `MC3` against the transmission FTIR spectrum, with the residual in fit. 

In [ ]:
Image("https://github.com/sarahshi/PyIRoGlass/raw/main/docs/_static/AC4_OL49_021920_30x30_H2O_a_modelfit.png")

The histogram figure shows the distribution of posterior probability densities, with the mean value displayed in the navy dashed line. The shaded region represents the 68% confidence interval around the value. 

In [ ]:
Image("https://github.com/sarahshi/PyIRoGlass/raw/main/docs/_static/AC4_OL49_021920_30x30_H2O_a_histogram.png")

The pairwise figure plots the posterior probability density distribution for the 16 fitting parameters of Equation 10, allowing for the visualization of covariance within the parameters. Accounting for covariance allows us to properly account for uncertainty. 

In [ ]:
Image("https://github.com/sarahshi/PyIRoGlass/raw/main/docs/_static/AC4_OL49_021920_30x30_H2O_a_pairwise.png")

The trace figure shows how the parameters evolve through MCMC sampling. 

In [ ]:
Image("https://github.com/sarahshi/PyIRoGlass/raw/main/docs/_static/AC4_OL49_021920_30x30_H2O_a_trace.png")

## LOG and NPZ

.log files record the performance of the MCMC algorithm through the samples, and the best parameters at each 10% increment. These are shown above. 

.npz files store all the best-parameters, sampled parameters, etc. in a ready-to-use `NumPy` format. 

We won't open these here, but these are quite useful to review! 

# Concentrations

We now want to convert all those peak heights (with uncertainties) to concentrations (with uncertainties), by applying the Beer-Lambert Law. We do so by using the `pig.calculate_concentrations` function, which takes in these parameters and samples over `N` samples for a secondary MCMC: 

- `Volatile_PH`: DataFrame with peak heights and their associated uncertainties; the output from `pig.calculate_baselines`
- `chemistry`: DataFrame of chemical data
- `thickness`: DataFrame of thickness data
- `export_path`: Output directory name, or `None` to prevent figure generation
- `N`: Number of Monte Carlo simulations to perform for uncertainty estimation, with default of 500,000
- `T`: Temperature in Celsius at which the density is calculated, with default of 25°C
- `P`: Pressure in bars at which the density is calculated, with default of 1 bar
- `model`: Choice of density model, with options of the default "LS" for Lesher and Spera (2015) and "IT" for Iacovino and Till (2019)

In [ ]:
concentrations_df = pig.calculate_concentrations(Volatile_PH, chemistry, thickness, export_path)

We're all done now! Let's display the `concentrations_df` DataFrame, which contains all results. 

In [ ]:
concentrations_df

There are a few things to note. Each column with the suffix `_MEAN` represents the mean value, `_BP` represents the best-parameter from MCMC, and `_STD` represents the standard deviation. We recommend the use of the `H2Ot_MEAN`, `H2Ot_STD`, `CO2_MEAN`, and `CO2_STD` columns. The columns with the suffix `_STN` show the signal-to-noise ratio of the NIR peaks, and the columns with the prefix `ERR_` just process this information, returning a `-` if the peaks are meaningful and a `*` if the signal is too low. 

Concentrations of $\mathrm{H_2O}$ depend on whether your sample is saturated or not. If your sample is unsaturated (marked by `H2Ot_3550_SAT=='-'`), the column `H2Ot_MEAN==H2Ot_3550_M`. If your sample is saturated (marked by `H2Ot_3550_SAT=='*'`), the column of `H2Ot_MEAN==(H2Om_1635_BP+OH_4500_M)`. The $\mathrm{H_2O_{t, 3550}}$ peak cannot be used, given potential nonlinearity in the Beer-Lambert Law. See the discussion of this handling of speciation in the paper. 

The column `Density` contains the densities used for the final concentration. The values between `Density` and `Density_Sat` will be different if the sample is saturated, showing the difference in densities when using variable concentrations of $\mathrm{H_2O_m}$. 

`Tau` and `Eta` calculate the compositional parameters required for determining molar absorptivity. All calculated molar absorptivities and their uncertainties (`Sigma_` prefix) from the inversion are provided in the DataFrame. 